In [ ]:
from fraude import load_cleaned_data
from pathlib import Path
from sklearn.feature_extraction import DictVectorizer

In [ ]:
project_path  = Path('..').resolve()

In [ ]:
cleaned_data = load_cleaned_data(project_path)
cleaned_data['week_day'] = cleaned_data['date'].apply(lambda date: date.strftime("%A"))
country_city = cleaned_data['city'].str.split(',')
cleaned_data['country'] = country_city.apply(lambda x: x[1] if len(x) > 1 else None)
cleaned_data['city'] = country_city.apply(lambda x: x[0] if len(x) > 0 else None)

In [ ]:
variables_independientes = ['week_day','amount','oldbalanceorg','newbalanceorig','city', 'country','type',
'card_type','exp_type','gender']
variable_dependiente = 'isfraud'

In [ ]:
cleaned_data[variables_independientes].head()

In [ ]:
from ydata_profiling import ProfileReport

In [ ]:
profile_report = ProfileReport(cleaned_data[variables_independientes + [variable_dependiente]].copy(), lazy=False)

In [ ]:
profile_report.to_notebook_iframe()

# Selección inicial de variables
De acuerdo con la matriz de correlaciones, teniendo en cuenta que la variable a predecir es is_fraud, notamos que las variables: type y amount son las que mayor correlación tienen con is_fraud, por lo tanto se procederá a graficar estas variables para identificar esto.

In [ ]:
import plotly.express as px

In [ ]:
fraude_inicial = cleaned_data[['type', 'amount', 'isfraud']]

In [ ]:
fraude_inicial

In [ ]:
px.violin(fraude_inicial.sample(n=10000), x='type', color='isfraud')
import pandas as pd
import plotly.express as px

#df = fraude_inicial.sample(n=1000)
df = fraude_inicial

# Calcular las fracciones de fraude y no fraude por tipo de transacción
df_grouped = df.groupby(['type', 'isfraud']).size().unstack(fill_value=0)
df_grouped['total'] = df_grouped.sum(axis=1)
df_grouped['fraude_fraccion'] = df_grouped[1] / df_grouped['total']
df_grouped['no_fraude_fraccion'] = df_grouped[0] / df_grouped['total']
df_grouped = df_grouped.reset_index()

# Crear el gráfico de barras apiladas con Plotly
fig = px.bar(df_grouped, x='type', y=['fraude_fraccion', 'no_fraude_fraccion'],
             labels={'value': 'Fracción de Transacciones', 'variable': 'Tipo de Transacción'},
             title='Fracción de Transacciones de Fraude y No Fraude por Tipo de Transacción',
             color_discrete_sequence=['red', 'green'])  # Puedes personalizar los colores

fig.update_layout(barmode='stack')  # Asegura que las barras estén apiladas

fig.show()


In [ ]:
import pandas as pd
import plotly.express as px

df = fraude_inicial

# Crear el histograma con Plotly Express
fig = px.histogram(df, x="amount", color="isfraud", barmode="overlay",
                   title="Distribución del Amount por Fraude",
                   labels={"amount": "Amount", "isfraud": "Es Fraude"})

# Mostrar el histograma
fig.show()
